In [3]:
from typing import Union, List, Any, Optional, Dict

import os
import re
import time
import json
import glob
import pickle
import random
import urllib
import requests

from bs4 import BeautifulSoup
from tqdm import tqdm
from pathlib import Path
from textblob import TextBlob

from transformers import BertTokenizer, BertModel

from utils import spar_utils
from utils import cleaning_utils

#### Example IFC class and description
* source: https://search-test.bsdd.buildingsmart.org/uri/buildingsmart/ifc-4.3/class/IfcWindow


We'll use: 
* TextBlob (NLTK's implementation of punkttokenizer under the hood) to split into sentences.
* SPaR.txt to predict which objects occur, some cleaning on top of this. 


In [5]:
label = "IfcWindow"

In [6]:
description = """
The window is a building element that is predominately used to provide natural light and fresh air. It includes vertical opening but also horizontal opening such as skylights or light domes. It includes constructions with swinging, pivoting, sliding, or revolving panels and fixed panels. A window consists of a lining and one or several panels. A window can:be a "free standing" window, contained in an IfcSpatialElement such as an IfcBuildingStorey. fill an opening, typically in a wall. The window will then have a FillsVoids attribute which uses the IfcRelFillsElement relationship to relate the IfcWindow with the IfcOpeningElement; be part of an element assembly, typically an IfcCurtainWall. The window will then have a Decomposes attribute which uses the the IfcRelAggregates relationship to relate the window with the assembly of elements;There are two main representations for window occurrences:IfcWindow entities that have a 3D rectangle 'Profile' shape representation defined. This profile can then be used to parametrically generate the geometry of a window. If not provided, the profile of the IfcOpeningElement can be used if the window fills an opening. The parameters are specified on the relating IfcWindowType that references IfcWindowLiningProperties and IfcWindowPanelProperties for each panel in the window; IfcWindow entities that are not parametrically generated and have only 'Brep', or 'SurfaceModel' geometry.In addition, an IfcWindow may commonly include a 'FootPrint' representation defining the 2D shape of the window and its swing.the window width and height the window opening direction (by the positive y-axis of the ObjectPlacement)The IfcWindowType specifies parameters which are common to all of its occurrences of IfcWindow:the partitioning type (single panel, double panel, tripel panel, more panels) the operation type (swing, tilt and turn, pivot revolve, fixed casement, etc.) the window panel hinge side (by using two different styles for right and left opening windows) the particular attributes for the lining by the IfcWindowLiningProperties the particular attributes for the panels by the  IfcWindowPanelPropertiesREFERENCE Definition according to ISO 6707-1 Construction for closing a vertical or near vertical opening in a wall or pitched roof that will admit light and may admit fresh air. NOTE The entity IfcWindowStandardCase has been deleted. Use an IfcWindow with a 'Profile' representation instead. The IfcWindow should also have an IfcWindowType with ParameterTakesPrecedence set to 'TRUE'. IFC4 CHANGE The attributes PredefinedType and OperationType are added, the applicable type object has been changed to IfcWindowType. HISTORY New entity in IFC1.0.
"""

In [7]:
# download SPaR.txt if required
from pathlib import Path
spartxt_path = Path("SPaR.txt/")
if not spartxt_path.exists():
    !git clone https://github.com/rubenkruiper/SPaR.txt.git

In [8]:
# kind of convoluted way to import spar.txt as a module from the parent directory
import imp
with open(spartxt_path.joinpath('spar_predictor.py'), 'rb') as fp:
    spar_predictor = imp.load_module(
        'spar_predictor', fp, 'SPaR.txt.spar_predictor.py',
        ('.py', 'rb', imp.PY_SOURCE)
    )

In [10]:
# TRAIN/LOAD
# - trains a model if needed, otherwise load from archive; 
# - best F1 on dev/validation in the paper is 80,96 trained on a GPU, CPU will be a bit lower ~77.x I think
te = spar_utils.TermExtractor(max_num_cpu_threads=1)

In [14]:
# use existing split into sentences functionality (uses textblob)
sentences = te.split_into_sentences(description)
# first 3 sents
sentences[:3]

['The window is a building element that is predominately used to provide natural light and fresh air.',
 'It includes vertical opening but also horizontal opening such as skylights or light domes.',
 'It includes constructions with swinging, pivoting, sliding, or revolving panels and fixed panels.']

In [15]:
objects = te.process_sentences(sentences)

In [16]:
objects

['The window',
 'a building element',
 'natural light',
 'fresh air',
 'vertical opening',
 'horizontal opening',
 'skylights',
 'light domes',
 'constructions',
 'swinging',
 'p',
 'sliding',
 'revolving',
 'panels',
 'fixed panels',
 'A window',
 'a lining',
 'one',
 'panels',
 'A window',
 'a',
 'free standing " window',
 'an IfcSpatialElement',
 'an IfcBuildingStorey',
 'an opening',
 'a wall',
 'The window',
 'a FillsVoids attribute',
 'the IfcRelFillsElement relationship',
 'the IfcWindow',
 'the IfcOpeningElement',
 'an element assembly',
 'an IfcCurtainWall',
 'The window',
 'a Decomposes attribute',
 'the',
 'the IfcRelAggregates relationship',
 'the window',
 'the assembly',
 'elements',
 'representations',
 'window occurrences',
 'IfcWindow entities',
 'a 3D rectangle',
 'Profile shape representation',
 'profile',
 'the geometry',
 'a window',
 'the profile',
 'the IfcOpeningElement',
 'the window',
 'an opening',
 'The parameters',
 'the relating IfcWindowType',
 'IfcWindow